In [212]:
import os, sys
sys.path = list(set([
    "../../lib/",
] + sys.path))
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Iterable, Any
from plotly import graph_objects as go, subplots as sp
from concurrent.futures import ProcessPoolExecutor as Exe
import networkx as nx
# import numba

from local.caching import load, save, save_exists
from local.figures import layout, xaxis_desc, yaxis_desc
# from txyl_common.biocyc_facade.pgdb import Pgdb, Dat, Traceable

In [230]:
def _circulant(n, k):
    return nx.Graph(nx.circulant_graph(n, [i+3 for i in range(k)]))

def _scale_free(n, k=4, seed:int=0, g=None):
    return nx.Graph(nx.barabasi_albert_graph(n, k, seed=seed, initial_graph=g))

seed = 125
np.random.seed(seed)

def get_n():
    return 45

sizes = []
def make_graph(depth: int, nlinks: list[int], split: int = 3) -> nx.Graph:
    assert depth <= len(nlinks), "not enough nlinks"
    if depth <= 0:
        n = get_n()
        sizes.append(n)
        return _scale_free(n, seed=np.random.randint(0, 9999))
    
    modules = [make_graph(depth-1, nlinks, split) for i in range(split)]

    G = nx.Graph()
    for m in modules:
        s = len(G)
        if s != 0:
            for _ in range(nlinks[depth-1]):
                a = np.random.randint(0, s)
                b = np.random.randint(s, s+len(m))
                G.add_edge(a, b)
        G.add_edges_from([(s+a, s+b) for a, b in m.edges])
    return G

G = make_graph(2, [15, 5])
    
pos = nx.kamada_kawai_layout(G)
# pos = nx.spring_layout(G)

In [247]:
# change in Q for each v in cluster if moved to other
def DeltaQ(G: nx.Graph, cluster, other):
    degrees_within = [G.degree[u] for u in cluster]
    degrees_out = [G.degree[u] for u in other]
    sum_in = np.sum(degrees_within)
    sum_out = np.sum(degrees_out)

    for v in cluster:
        neighbours = [u for u in G.neighbors(v)]
        if len(other) > 0 and all(u in cluster for u in neighbours): continue
        edges_within = [u for u in neighbours if u in cluster]
        edges_out = [u for u in neighbours if u not in cluster]

        vsum_in = (sum_in - G.degree[v])
        vsum_out = sum_out
        Q = (len(edges_out) - len(edges_within)) / len(G.edges)
        Q -= G.degree[v] * (vsum_out - vsum_in) / (2 * len(G.edges)**2)

        yield Q, v

def ToGive(G: nx.Graph, cluster: set[int], other: set[int]):
    if len(cluster)==0: return [], []
    to_give = sorted(DeltaQ(G, cluster, other), key=lambda t: t[0], reverse=True)
    if len(cluster) == len(G):
        cut = 1
        force = True
    else:
        cut = max(1, len(G)//100)
        # cut = 1
        force = False 

    qs, vs = [], []
    for q, v in to_give:
        if len(qs) >= cut: break
        if not force and q <= 0: continue
        qs.append(q)
        vs.append(v)
    return qs, vs

def Partition(G: nx.Graph):
    if len(G.edges) == 0: return None

    cluster = {v for v in G.nodes}
    other = set()

    i = 0
    delta_q = 0
    _break = False
    while not _break:
        qs, to_give = ToGive(G, cluster, other)
        if len(to_give) == 0: break
        if len(to_give) >= len(cluster):
            cut = len(cluster)-1
            qs = qs[:cut]
            to_give = to_give[:cut]
            _break = True
        delta_q += np.sum(qs)
        cluster = cluster.difference(to_give)
        other = other.union(to_give)
        i += 1
    # print(f"iterations: {i}, deltaQ: {delta_q}, {len(cluster)}, {len(other)}")
    if len(other) == 0: return None

    def _partition(cluster: set[int]):
        g = nx.Graph()
        g.add_nodes_from(cluster)
        for v, u in G.edges:
            if v in cluster and u in cluster:
                g.add_edge(v, u)
        return g

    return _partition(cluster), _partition(other), delta_q

def Cluster(G: nx.Graph):
    _original = G
    clusters = [{v for v in G.nodes}]
    current_modularity = nx.algorithms.community.modularity(G, clusters)
    def _is_same(a: set, b: set):
        return next(iter(a)) in b
    
    def _cluster(G: nx.Graph):
        leaf = [v for v in G.nodes]
        if len(G) <= 1: return leaf
        part = Partition(G)
        if part is None: return leaf

        a, b, dq = part
        ab = {v for v in G.nodes}
        cluster_a = {v for v in a}
        cluster_b = {v for v in b}
        if len(cluster_a) > len(cluster_b):
            inplace, new = cluster_a, cluster_b
        else:
            inplace, new = cluster_b, cluster_a

        nonlocal clusters, current_modularity
        snapshot = []
        for c in clusters:
            if  _is_same(c, ab): 
                snapshot.append(inplace)
            else:
                snapshot.append(c)
        snapshot.append(new)
        new_mod = nx.algorithms.community.modularity(_original, snapshot)
        if new_mod < current_modularity: return leaf

        current_modularity = new_mod
        clusters = snapshot

        return _cluster(a), _cluster(b), dq
    
    return _cluster(G), clusters, current_modularity

print(f"N={len(G)}")
tree, clusters, mod = Cluster(G)

N=405


In [238]:
def View(tree: tuple|list, q_cutoff=None):
    iterations = []
    delta_q = [0]
    current_groups: list[set[int]] = []

    def _view(_tree):
        if isinstance(_tree, list):
            leaf = set(_tree)
            return leaf
        
        a, b, q = _tree
        return _view(a).union(_view(b))

    current_groups: list[set[int]] = [_view(tree)]
    iterations.append(current_groups)

    def _is_same(a: set, b: set):
        return next(iter(a)) in b
    
    todo: list[tuple|list] = [tree]
    while len(todo)>0:
        _tree = todo.pop(0)
        if isinstance(_tree, list): continue

        a, b, q = _tree
        cluster_a = _view(a)
        cluster_b = _view(b)

        if len(cluster_a) > len(cluster_b):
            inplace, new = cluster_a, cluster_b
        else:
            inplace, new = cluster_b, cluster_a

        ab: set[int] = cluster_a.union(cluster_b)
        snapshot = []
        for c in current_groups:
            if  _is_same(c, ab): 
                snapshot.append(inplace)
            else:
                snapshot.append(c)
        snapshot.append(new)
        
        current_groups = snapshot
        # if q > 0:
        if q_cutoff is None or q > q_cutoff:
            iterations.append(snapshot)
            delta_q.append(q)
            todo.append(a)
            todo.append(b)

    return iterations, delta_q


all_iterations, all_delta_q = View(tree)
iterations, delta_q = View(tree, 0.12)
dqs = [f"{q:0.3f}" for q in delta_q]
# print(f"{dqs}")
print(len(delta_q))

9


In [239]:
fig = sp.make_subplots(
    rows=1, cols=1, shared_xaxes=True, shared_yaxes=False, horizontal_spacing=0.05, vertical_spacing=0.02,
)
# aggregate_q = [0]
# sumq = 0
# i_at_maxq, maxq = 0, 0
# for i, q in enumerate(delta_q):
#     sumq+=q
#     if sumq > maxq:
#         i_at_maxq = i
#         maxq = sumq
#     aggregate_q.append(sumq)
# iterations = all_iterations[:i_at_maxq]
# print(f"{i_at_maxq}")
thresh = -99
fig.add_trace(
    go.Scatter(
        # x = [i for i, q in enumerate(aggregate_q)], y = [q for i, q in enumerate(aggregate_q)],
        x = [i for i, q in enumerate(all_delta_q) if q > thresh], y = [q for i, q in enumerate(all_delta_q) if q > thresh],
        mode="markers+lines",
        marker=dict(
            size=5,
            # symbol="circle-open",
            # color=_color,
        ),
        line=dict(width=2),
        showlegend=False,
        # name=_name,
    ),
    row=1, col=1,
)
_layout:Any = layout.copy()
_layout.update(dict(
    height=700,
    # yaxis=dict(title="delta Q", range=(0, 0.6), **yaxis_desc),
    yaxis=dict(title="delta Q", **yaxis_desc),
    # yaxis2=dict(**yaxis_desc),
    # yaxis3=dict(title="Est. Av. Min. Path Length", **yaxis_desc),
    # yaxis4=dict(**yaxis_desc),

    # xaxis2=dict(**yaxis_desc),
    # xaxis3=dict(title="Graph Size", **yaxis_desc),
    # xaxis4=dict(title="Graph Size", **yaxis_desc),
))
fig.update_layout(go.Layout(_layout))
fig.show()

In [246]:
_layout: Any = layout.copy()
no_col = 'rgba(0, 0, 0, 0)'
axis_desc: dict = dict(
    linecolor=no_col, gridcolor=no_col, zerolinecolor=no_col, zerolinewidth=0,
    visible=False, range=(-1.1, 1.1), autorange=False,
)

_layout.update(dict(
    width=700, height=700,
    xaxis=axis_desc, yaxis=axis_desc,
))
FRAME_RATE = 2
_frame_period = int(round(1000/FRAME_RATE))
def _plot(G: nx.Graph, pos: dict, groups: Iterable[set]):
    # colors = "#0093CD, #FF9447, #90EA85".split(", ")
    colors = ["#e60049", "#0bb4ff", "#50e991", "#e6d800", "#9b19f5", "#ffa300", "#dc0ab4", "#b3d4ff", "#00bfa0"]
    glookup = {}
    for i, g in enumerate(groups):
        for v in g:
            glookup[v] = i%len(colors)
    cs = [colors[glookup[v]] if v in glookup else "#000000" for v in G.nodes]
    n_plot = go.Scatter(
            x=[x for i, (x, y) in enumerate(pos.values())],
            y=[y for i, (x, y) in enumerate(pos.values())],
            text=[f"{v}" for v in G.nodes],
            # hoverinfo="text",
            mode="markers", showlegend=False,
            marker=dict(
                color=cs,
            )
        )

    edges = []
    for a, b in G.edges:
        edges += [(pos[a]), (pos[b]), (None, None)]
    e_plot = go.Scatter(
        x=[x for x, y in edges], y=[y for x, y in edges],
        mode="lines", showlegend=False,
        line=dict(width=1, color="rgba(0, 0, 0, 0.2)"),
    )
    plots = (e_plot, n_plot)
    return plots

frames = []
sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Iteration:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 0, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}
for i, clusters in enumerate(iterations):
    slider_step = {
        "args": [
            [i],
            {"frame": {"duration": 0, "redraw": False},
            "mode": "immediate",
            "transition": {"duration": 0}}
        ],
        "label": i,
        "method": "animate"
    }
    sliders_dict["steps"].append(slider_step)
    frames.append(go.Frame(data=_plot(G, pos, clusters), name=str(i)))

fig = go.Figure(
    data=_plot(G, pos, iterations[-1]),
    layout=go.Layout(
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": _frame_period, "redraw": False},
                                        "fromcurrent": True, "transition": {"duration": 0,
                                                                            "easing": "quadratic-in-out"}}],
                        "label": "Play",
                        "method": "animate"
                    },
                    {
                        "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                        "mode": "immediate",
                                        "transition": {"duration": 0}}],
                        "label": "Pause",
                        "method": "animate"
                    }
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top"
            }
        ],
        sliders = [sliders_dict],
        **_layout
    ),
    frames=frames,
)

fig.show()